In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# 파일 경로 설정
file_path = '/content/drive/MyDrive/우/Data4.xlsx'  # 엑셀 파일의 경로를 여기에 입력하세요.

# 엑셀 파일 불러오기
data = pd.read_excel(file_path)

# 데이터 확인
print(data.head())


                                            SENTENCE  \
0                                          지금 배달되나요?   
1                                          아 네 배달됩니다   
2                       짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?   
3  특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다   
4                          전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?   

                       label_idx  
0                   487,3103,265  
1                            265  
2                           2960  
3  2895,2509,4189,1117,4190,3974  
4                      3974,2895  


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
tokenized_data = tokenizer(data['SENTENCE'].tolist(), padding=True, truncation=True, max_length=512)


(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

(…)lue/roberta-small/resolve/main/vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

(…)oberta-small/resolve/main/tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

(…)all/resolve/main/special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [5]:
import numpy as np

def convert_to_multilabel(labels, num_labels):
    multilabels = np.zeros(num_labels)
    for label in labels:
        multilabels[label] = 1
    return multilabels

unique_labels = set()
for label_list in data['label_idx']:
    labels = [int(label) for label in label_list.split(',')]
    unique_labels.update(labels)

max_label_idx = max(unique_labels)
data['multilabel'] = data['label_idx'].apply(lambda x: convert_to_multilabel([int(label) for label in x.split(',')], max_label_idx + 1))


In [6]:
from transformers import TFRobertaForSequenceClassification
import tensorflow as tf

model = TFRobertaForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=max_label_idx + 1)

# 이 부분에서 모델 컴파일 및 학습 과정을 추가합니다.


(…)e/roberta-small/resolve/main/config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
from transformers import TFRobertaForSequenceClassification, AutoTokenizer
import tensorflow as tf
import numpy as np
import pandas as pd

# 레이블 데이터를 멀티 레이블 형식으로 변환 (이전 단계에서 설명한 함수 사용)
# ...

# 토크나이저 로드 및 데이터 토크나이징
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
tokenized_data = tokenizer(data['SENTENCE'].tolist(), padding=True, truncation=True, max_length=512)

# 모델 로드
model = TFRobertaForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=max_label_idx + 1)

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# NumPy 배열을 TensorFlow 텐서로 변환
tokenized_data_tensors = {key: tf.convert_to_tensor(value) for key, value in tokenized_data.items()}

# 멀티 레이블 데이터를 하나의 NumPy 배열로 변환한 후 TensorFlow 텐서로 변환
multilabels_array = np.stack(data['multilabel'].values)
multilabels_tensor = tf.convert_to_tensor(multilabels_array)

# 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((tokenized_data_tensors, multilabels_tensor))
train_dataset = train_dataset.shuffle(len(data)).batch(32)

# 모델 학습
model.fit(train_dataset, epochs=20)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/20
492/492 [==============================] - 164s 279ms/step - loss: 0.0724 - accuracy: 0.0871
Epoch 2/20
492/492 [==============================] - 126s 256ms/step - loss: 0.0047 - accuracy: 0.1141
Epoch 3/20
492/492 [==============================] - 128s 259ms/step - loss: 0.0034 - accuracy: 0.1141
Epoch 4/20
492/492 [==============================] - 127s 257ms/step - loss: 0.0031 - accuracy: 0.1141
Epoch 5/20
 96/492 [====>.........................] - ETA: 1:41 - loss: 0.0030 - accuracy: 0.1178